<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Sentiment/merge_Clarifai_Sentiment_Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#da cambiare:
#nome_ristorante_sort con il sort 
#users_restaurant_id (link condivisibile del csv clarifai)
#sentiment_id (link condivisibile del csv sentiment)

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import pandas as pd
import csv

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nome_ristorante_sort = 'CulinaryDropout'

# users_restaurant_id = id del file csv del ristorante (si trova in data > face_data > Clarifai) > (nomeRistorante) > (nomeRistorante)__NoDuplicati_Random_UsersData.csv)
#File dove ci sono id e recensione
users_restaurant_id = '1alWrWOEzAwrThey-B7qaR2CTGAfxEFPP'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_restaurant_id})
download.GetContentFile('%s' % nome_ristorante_sort + '_NoDuplicati_Random_UsersData.csv')

#leggo il file csv del ristorante
clarifai = pd.read_csv('%s' % nome_ristorante_sort + '_NoDuplicati_Random_UsersData.csv')

# sentiment_id = id del file csv del ristorante (si trova in data > Sentiment > Sentiment Analisys > (nomeRistorante) > (nomeRistorante)_Random_ReviewSentiment.csv)
sentiment_id = '1lV9wxvN4zYf2-VxpwoYVBncCpZQPxmTv'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': sentiment_id})
download.GetContentFile('%s' % nome_ristorante_sort + '_Random_ReviewSentiment.csv')

#leggo il file csv del ristorante
sentiment = pd.read_csv('%s' % nome_ristorante_sort + '_Random_ReviewSentiment.csv')

#foto = pd.read_csv('%s' % nome_ristorante + '_FotoRandom.csv')
merged = sentiment.merge(clarifai,how = 'left',on='id')

df = merged.rename(columns={"id": "user_id"})

columnsTitles=["user_id","age","gender","ethnicity","review_sentiment"]
df1=df.reindex(columns=columnsTitles)

#csv contenente user_id e review
df1.to_csv('%s' % nome_ristorante_sort + '_Analisys.csv', index=False)

# making data frame from csv file 
nba = pd.read_csv('%s' % nome_ristorante_sort + '_Analisys.csv')

# replacing na values in college with No college 
nba["age"].fillna("Not recognized", inplace = True) 
nba["gender"].fillna("Not recognized", inplace = True) 
nba["ethnicity"].fillna("Not recognized", inplace = True) 

#csv contenente user_id e review
nba.to_csv('%s' % nome_ristorante_sort + '_Analisys.csv', index=False)

folder_id = '1K5kXpkOhp4CkyUnOfOdaLixbCklxi9yr'    #cartella dove metto il csv
file_metadata = {'title': '%s' % nome_ristorante_sort + '_Analisys.csv',"parents":[{"id": folder_id,"kind":"drive#childList"}]}
folder = drive.CreateFile(file_metadata)
folder.SetContentFile('%s' % nome_ristorante_sort + '_Analisys.csv') #The contents of the file
folder.Upload()